<a href="https://colab.research.google.com/github/ab-sa/Statistical-Machine-Learning-2/blob/main/Lecture4_LASSO_Ridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold
#import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

Import Credit data

In [ ]:
Credit = pd.read_csv('Credit.csv')
#Insurance = pd.read_csv('Insurance.csv')
print('Dimension of the data: ' + str(Credit.shape))
Credit.head()

Data preprocessing: splitting, standardizing, etc.

In [ ]:
Credit_dumms = pd.get_dummies(Credit)
y = Credit_dumms['Balance']
X = Credit_dumms.drop(['ID', 'Balance'], axis=1)

X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

std_scale = StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std  = std_scale.transform(X_test)

**Ridge**: A linear model with a Ridge penalty term (norm 2)

First: with a fixed lambda (alpha) on a samll set of features:

In [ ]:
X_temp = X_train[['Limit', 'Rating', 'Cards', 'Age', 'Education']]
std_scale_temp = StandardScaler().fit(X_temp)
X_temp_std = std_scale_temp.transform(X_temp)

LMRidge = Ridge(alpha=1.0)
print(LMRidge.fit(X_temp, y_train).coef_)
print(LMRidge.fit(X_temp_std, y_train).coef_)

Second: Find the optimal value for alpha (lambda) on all features:

In [ ]:
# model evaluation: coss validation
CV10 = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
LMRidgeCV = RidgeCV(alphas=np.arange(0, 1, 0.01), cv=CV10, scoring='neg_mean_squared_error')
LMRidgeCV.fit(X_train_std, y_train)
# summarize chosen configuration
print('alpha: %f' % LMRidgeCV.alpha_)

Fit a Ridge model with the optimal alpha (lambda):

In [ ]:
LMRidge = Ridge(alpha = LMRidgeCV.alpha_)
LMRidge.fit(X_train_std, y_train)
mean_squared_error(y_test, LMRidge.predict(X_test_std))
print(LMRidge.coef_)

LASSO: Fit a linear model with a LASSO penalty term (norm 1):

In [ ]:
lassocv = LassoCV(alphas = np.arange(0.5, 2, 0.01), cv = CV10)
lassocv.fit(X_train_std, y_train)
print(lassocv.alpha_)

LMLasso = Lasso()
LMLasso.set_params(alpha=lassocv.alpha_)
LMLasso.fit(X_train_std, y_train)
mean_squared_error(y_test, LMLasso.predict(X_test_std))
print(LMLasso.coef_)